# Workshop ULLA2023: QSAR modelling

In this workshop you will design your own compounds and predict their properties using QSAR (Quantitative Structure-Activity Relationship) modelling. You will learn how to build and evaluate QSAR models and how to use them to predict the properties of new compounds.

# Install QSPRpred

 <a href="https://github.com/CDDLeiden/QSPRPred">QSPRPred</a> is an open-source software libary for building **Quantitative Structure Property Relationship (QSPR)** models developed by Gerard van Westens Computational Drug Discovery group. It provides a unified interface for building QSPR models based on different types of descriptors and machine learning algorithms.
 In this tutorial, you will use QSPRPred to build QSAR models for your chosen target protein. You can install QSPRPred using the following command:

In [ ]:
%pip install git+https://github.com/CDDLeiden/QSPRPred.git@BOO-2023

# Create your dataset

First, you need to create a dataset for your target protein. The data will be collected from <a href="https://github.com/CDDLeiden/Papyrus-scripts">Papyrus</a>. 

The Papyrus dataset is a highly curated compilation of bioactivity data intended for modeling in drug discovery. Apart from the bioactivity data contained in the [ChEMBL database](https://www.ebi.ac.uk/chembl/), the Papyrus dataset contains binary data for classification tasks from the [ExCAPE-DB](https://solr.ideaconsult.net/search/excape/), and bioactivity data from a number of kinase-specific papers. The Papyrus dataset consists of almost 60M compound-protein pairs, representing data of around 1.2M unique compounds and 7K proteins across 499 different organisms.

The aggregated bioactivity data is standardized, repaired, and normalized to form the Papyrus dataset. The Papyrus dataset contains "high quality" data associated with pChEMBL values that represent exact bioactivity values measured and associated with a single protein or complex subunit. A pChEMBL value is a canonical activity metric defined as $-log_{10}(molar IC_{50}, XC_{50}, EC_{50}, AC_{50}, Ki, Kd, or potency)$. Moreover, "low and medium quality" data that is associated with potentially multiple proteins or a homologous single proteins and/or only approximate bioacivity values or binary active/inactive label is also included in the Papyrus dataset, but is not used in this workshop.

Here I've already made a pre-selection from the Papyrus dataset (including only the data for all the proteins used in the workshop) to shorten the data collection process. You still need to collect the data from the dataset for your specific target protein. You can do this by replacing the accession number in the code below with your target protein accession number. You can find the accession number of your target protein in the <a href="https://www.uniprot.org/">UniProt</a> database.


In [ ]:
# read in all data

import pandas as pd
df = pd.read_csv('data/papyrus_data.tsv', sep='\t')

# filter data for target of interest
MY_TARGET = 'P21802' # REPLACE WITH YOUR TARGET ACCESSION

df = df[df['accession'] == MY_TARGET]

# keep only high quality data
df = df[df['Quality'].isin(['High'])] # if you want to keep also medium and low quality data, remove this line

# Create molecule table for visualization
from qsprpred.data.data import MoleculeTable

mt = MoleculeTable(df=df, name=MY_TARGET, store_dir='data')

mt.getDF()

## Preparing data for modelling

After you have collected the data for your target protein, you need to prepare the data for modelling.
First we will specify the name of the target property (pchembl_value_Median) and the model task (REGRESSION) to 
create our QSPRdataset object. The QSPRdataset object is a wrapper around the pandas dataframe that contains the data for modelling.

In [ ]:
from qsprpred.models.tasks import TargetTasks
from qsprpred.data.data import QSPRDataset

target_props=[{
                "name": "pchembl_value_Median", # name of the target column in the dataset
                "task": TargetTasks.REGRESSION, # specify the task type (SINGLECLASS, MULTICLASS, REGRESSION)
                }]

# Create a QSPRDataset instance used for training and evaluation of QSPR models
dataset = QSPRDataset.fromMolTable(mt, target_props=target_props)
dataset.targetProperties

Next, we will use the `prepareDataset` method to prepare the data for modelling. This method will perform the following steps:

1. Standardize the SMILES sequences
2. Split the data into training and test sets (80/20 split)
3. Calculate the descriptors (Morgan Fingerprints) as discussed in the lecture (See also image below)


![descriptors](figures/descriptors.png)

In [ ]:
from qsprpred.data.utils.descriptorsets import FingerprintSet
from qsprpred.data.utils.descriptorcalculator import MoleculeDescriptorsCalculator
from qsprpred.data.utils.datasplitters import randomsplit

# Calculate MorganFP
feature_calculator = MoleculeDescriptorsCalculator(descsets = [FingerprintSet(fingerprint_type="MorganFP", radius=3, nBits=2048)])

# Do a random split for creating the train (80%) and test set (20%)
rand_split = randomsplit(0.2)

# calculate compound features and split dataset into train and test
dataset.prepareDataset(
    split=rand_split,
    feature_calculators=[feature_calculator]
)

# Let's have a look at the features of the first five compounds
print(f"Number of features (bits in the MorganFP): {len(dataset.getFeatures()[0].columns)}")
display(dataset.getFeatures()[0].head())

print(f"Number of samples train set: {len(dataset.y)}")
print(f"Number of samples test set: {len(dataset.y_ind)}")

# Let's save the dataset for later
dataset.save()

# Data Visualization

Now that we have prepared the data for modelling, we can visualize the data to get a better understanding of what is in our dataset. We will first plot a histogram of the target property values to get an idea of the distribution of the data.

In [ ]:
# create histogram of pchembl values in the dataset
import seaborn as sns
sns.histplot(dataset.getDF()['pchembl_value_Median'], bins=20)

As the goal of this workshop is to design your own compounds, we will also have a look at the compounds with the highest affinity for your target. We will use the `draw.MolsToGridImage` method to draw the compounds with the highest pChEMBL values. You can change the number of compounds to draw by changing the `NUM_COMPOUNDS` variable.

In [ ]:
# Visualize the compounds with the highest pchembl values in the dataset
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import MolFromSmiles

# number of compounds to show
NUM_COMPOUNDS = 10

# Sort the dataset by pchembl value
dataset_sorted = dataset.getDF().sort_values(by='pchembl_value_Median', ascending=False)
 
# show average pchembl value per scaffold and the count of compounds per scaffold
Draw.MolsToGridImage([MolFromSmiles(smiles) for smiles in dataset_sorted[:NUM_COMPOUNDS].SMILES], molsPerRow=5, subImgSize=(200,200), legends=[f"{dataset_sorted['pchembl_value_Median'][idx]:.2f}" for idx in dataset_sorted[:NUM_COMPOUNDS].index])

Although molecules with a high pChEMBL value can be a good starting point for designing your own compound, it can be difficult to know what to changes to make to improve the affinity. In drug discovery, chemists often explore the structure-activity relationship (SAR) of a compound by making small changes to a core structure (`scaffold`), such as replacing a substituent or adding a functional group. We can use this information by extracting the scaffolds from the compounds and finding the scaffolds with on average the highests pChEMBL values to better understand the SAR of the compounds in our dataset. 

Here we will visualize those scaffolds. You can change the number of scaffolds to draw by changing the `NUM_SCAFFOLDS` variable and the `MIN_COMPOUNDS` variable to change the minimum number of compounds that should contain the scaffold. Note, the numbers under the scaffolds are 
*{index}: {average pChEMBL value} ({number of compounds that contain the scaffold})*.

In [ ]:
from qsprpred.data.utils.scaffolds import Murcko, BemisMurcko

# Show top n scaffolds with at least x compounds
NUM_SCAFFOLDS = 10
MIN_COMPOUNDS = 5

dataset.addScaffolds([Murcko()])

# get average pchembl value per scaffold
scaffolds = dataset.getDF().groupby('Scaffold_Murcko')['pchembl_value_Median'].mean().sort_values(ascending=False)
scaffolds = scaffolds.rename('Average pchembl value')

# add the number of compounds per scaffold
scaffolds = pd.concat([scaffolds, dataset.getDF().groupby('Scaffold_Murcko')['pchembl_value_Median'].count()], axis=1)
scaffolds = scaffolds.rename(columns={'pchembl_value_Median': 'Count'})

# Drop scaffolds with less than MIN_COMPOUNDS compounds
scaffolds = scaffolds[scaffolds['Count'] > MIN_COMPOUNDS]
 
# show average pchembl value per scaffold and the count of compounds per scaffold
Draw.MolsToGridImage([MolFromSmiles(scaffold) for scaffold in scaffolds.index[:NUM_SCAFFOLDS]], molsPerRow=5, subImgSize=(200,200), legends=[f"{idx}: {scaffolds['Average pchembl value'][scaffold]:.2f} ({scaffolds['Count'][scaffold]})" for idx, scaffold in enumerate(scaffolds.index[:NUM_SCAFFOLDS])])

After you have studied the promising scaffolds, you can select one of the scaffolds to design your own compound. Let's have a look at what molecules that have been tested that contain this scaffold. You can do this by changing the `SCAFFOLD_INDEX` variable to the scaffold that you want to see the compounds of. The index of the scaffold is the first number under the scaffold image.

In [ ]:
# Show all compounds from from scaffold N.

SCAFFOLD_INDEX = 0 # REPLACE WITH YOUR SCAFFOLD INDEX

scaffold = scaffolds.index[SCAFFOLD_INDEX]

# get all compounds from scaffold
scaffold_df = dataset.getDF()[dataset.getDF()['Scaffold_Murcko'] == scaffold]

# sort compounds by pchembl value
scaffold_df = scaffold_df.sort_values(by='pchembl_value_Median', ascending=False)

# visualize compounds
Draw.MolsToGridImage([MolFromSmiles(smiles) for smiles in scaffold_df.SMILES], molsPerRow=5, subImgSize=(200,200), legends=[f"{scaffold_df['pchembl_value_Median'][idx]:.2f}" for idx in scaffold_df.index])

# Training a ML model

After preparing the data for modelling, we can train a machine learning model to predict the target property.


First, we will specify the machine learning algorithm that we want to use to train our model. For this workshop we will use the Partial Least Squares Regression model from <a href="https://scikit-learn.org/stable/">scikit-learn</a> as this model is fast to train. However, the model algorithm could easily be changed to a different model by replacing the alg argument with a different model from scikit-learn. You can find a list of models in the <a href="https://scikit-learn.org/stable/supervised_learning.html">scikit-learn documentation</a>.

See the image below for an overview of the steps involved in training a machine learning model.
1. We will first optimize the hyperparameters of the model using the bayesian optimization (See <a href="https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f"> this tutorial </a> for an explanation) for five trials. Normally, you would use more trials to find the best hyperparameters, but this can take a long time, so we will limit the number of trials for this workshop. You can change the number of trials by changing the `n_trials` variable.
2. Using the `evaluate` function, we will train the model using the optimized hyperparameters on the training data using cross-validation.
3. Also in the `evaluate` function, we will evaluate the model on the test data using the optimized hyperparameters.
4. Finally we will train the model on all the data with the `fit` method to use it for predicting the activities of new compounds.




![modelling](figures/modelling_workflow.png)

In [ ]:
from qsprpred.models.models import QSPRsklearn
from sklearn.cross_decomposition import PLSRegression
from qsprpred.models.hyperparam_optimization import OptunaOptimization

# This is an SKlearn model, so we will initialize it with the QSPRsklearn class
model = QSPRsklearn(base_dir = '.', data=dataset, alg = PLSRegression, name='PLS_REG')

# We will first optimize the hyperparameters (n_components and scale) through bayes optimization
# the best hyperparameter combination will be saved in PLS_REG_params.json
search_space_bs = {"n_components": ["int", 1, 30], "scale": ["categorical", [True, False]]}
bayesoptimizer = OptunaOptimization(scoring = model.score_func, param_grid=search_space_bs, n_trials=5)
best_params = bayesoptimizer.optimize(model)

#Then we will evaluate the performance of the best model using the independent test set
_ = model.evaluate()

# Finally, we need to fit the model on the complete dataset if we want to use it further
# model is saved under qspr/models/PLS_REG.json
model.fit()

After training a model, you can visualize the predictions on the evaluation folds of the cross validation and the test set to get an impression of the performance of the model by creating a scatter plot of the predicted vs. the actual values. Also the R2 and RMSE values are printed to get a better idea of the performance of the model (cv stands for cross-validation and ind for the test set).

- **Coefficient of determination (**$R^{2}$ **score)**: Represents the portion of variance of the target variable that has been explained by the independent variables (features) in the model. $R^{2}$ score varies between 1.0 (best score) and minus infinite, where 0.0 represents a model that always predicts the average target variable. As the variance is dataset-dependent, it might not be a meaningful metric to compare between datasets. When dealing with linear regression, and model fitting and evaluation are performed on a single dataset, $R^{2}$ is equivalent to the square of the Pearson correlation coefficient, described below, and can be noted as $r^{2}$.,
- **Root mean square error (RMSE)**: It is the square root of the MSE and represents the standard deviation of the prediction errors with respect to the line of best fit. RMSE is a measure of accuracy and it cannot be applied to compare  between datasets, as it is scale-dependent. It varies between infinite and 0.0 (best).,

In [ ]:
from qsprpred.plotting.regression import CorrelationPlot

plt = CorrelationPlot([model])
axes, summary = plt.make(save=False, property_name='pchembl_value_Median')
axes[0]

print(summary)

# Make predictions for your own compounds

Now that you have trained a model, you can use it to predict the activities of your own compounds. You can do this by replacing the SMILES sequences in the `smiles` list with your own SMILES sequences. To create your own compounds you can draw them using <a href=https://molview.org>MolView</a> and go to the `Tools` tab and click on `Information card`. You can then copy the SMILES sequence from the `Information card` and paste it in the `list_of_smiles`. The model prediction will then be printed for each compound.

In [ ]:
# replace with your own compounds
list_of_smiles = ['OCCc1ccn2cnccc12',
                  'C1CC1Oc1cc2ccncn2c1',
                  'CNC(=O)c1nccc2cccn12'] # REPLACE WITH YOUR OWN COMPOUNDS

# make predictions with the model
predictions = model.predictMols(list_of_smiles)

# show molecules with predicted values using rdkit
from rdkit import Chem
from rdkit.Chem import Draw
 
mols = [Chem.MolFromSmiles(smi) for smi in list_of_smiles]
Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(200, 200), legends=[f'{pred[0]:.3f}' for pred in predictions])